1.	过滤关键词中其他字符，只保留汉字re.sub('[^\u4e00-\u9fa5]', '', 'a,.b,123.阿阿')
2.	对关键词本身进行分词、国家名称映射、国家名称统计，从大到小排序，取最大值对应的关键字。   
3.	通过wiki百科对英文关键词进行检索
4.	如果有道可用，则先用有道进行翻译，判断有道反馈的信息中是否包含这个关键词的国家信息，有则提出。否则通过百度百科对翻译出的中文关键字进行检索，提取出最有可能的国家名称。
5.	对2、3、4流程得出的结果进行优先级处理，2最高，3、4随机
6.	将上5步得出的结果做成键值对形式的词表，再进行关键字与词表进行映射匹配
7.	人工复查

- 分词： segment_list = jieba.cut_for_search(text)
- 国家名称映射：segment_list = list(filter(lambda s: True if s in countries else False, segment_list))
- 统计 counted_list = dict(Counter(list_all))
- 排序（从大到小）sorted_list = sorted(counted_list.items(), key=operator.itemgetter(1), reverse=True)

+ wiki_zh-cn_api
```
https://w.bk.wjbk.site/w/api.php?
format=json&action=query&generator=search&gsrnamespace=0&gsrlimit=10&prop=pageimages|extracts&pilimit=max
&exintro&explaintext&exsentences=1&exlimit=max&origin=*&gsrsearch={keyword}
```

+ baike_api
```
http://baike.baidu.com/api/openapi/BaikeLemmaCardApi?scope=103&format=json&appid=379020&bk_key={keyword}`
```

In [32]:
import requests
import jieba
import re
import time
import random
import hashlib
import gzip
from urllib.parse import quote
from lxml.etree import HTML
from fake_useragent import UserAgent
from collections import Counter
from scrapy.selector import Selector
import operator
import json
from json import JSONDecodeError

In [2]:
countries_json_file = r'E:\文档\SpiderProject\TestSpider\全球国家.json'

with open(countries_json_file, 'r', encoding='utf-8') as f:
    countries = json.load(f)
countries = set(countries)
ua = UserAgent()

In [3]:
def is_chinese(text):
    '''
        判断是否为中文
    '''
    return True if re.findall('([\u4e00-\u9fa5]+)', text) else False

In [4]:
def get_wiki(keyword):
    '''
        通过关键词，获取wiki反馈的html信息
    '''
    # wiki_api = 'https://w.bk.wjbk.site/wiki/{}'
    wiki_api = 'https://baike.baidu.com/item/{}'
    headers = {
        'User-Agent': ua.random,
    }
    url = wiki_api.format(quote(keyword), 'utf-8')
    r = requests.get(url, headers=headers)
    r.encoding = r.apparent_encoding
    return r.text

In [5]:
def get_tags_text_by_xpath(text, xpath):
    '''
        通过xpath获取标签的文本信息
    '''
    # html = HTML(text)
    # tags_text_list = html.xpath(xpath)
    selector = Selector(text=text).xpath(xpath)
    tags_text_list = selector.extract()
    return tags_text_list

In [6]:
def get_text_segment(text):
    '''
        结巴分词
    '''
    if isinstance(text, str):
        return list(jieba.cut_for_search(text))
    elif isinstance(text, list):
        segment_list = list()
        for w in text:
            segment_list.extend(jieba.cut_for_search(w))
        return segment_list

In [7]:
def get_list_most_element(list_all):
    '''
        返回一个列表中元素个数最多的那个元素
    '''
    # 进行统计
    counted_list = dict(Counter(list_all))
    # 进行排序
    sorted_list = sorted(counted_list.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_list[0][0]

In [8]:
def get_country_from_text(data):
    '''
        从字符串中提取出出现频率最高国家名称
    '''
    # 对字符串进行分词
    segment_list = get_text_segment(data)
    # 过滤非国家名称字符串
    segment_list = list(filter(lambda s: True if s in countries else False, segment_list))
    # 如果列表中含有国家名称则进行统计排序，返回频率最高的国家名称
    if segment_list:
        country = get_list_most_element(segment_list)
        return country
    return ''

In [38]:
def get_html(url, encoding='utf-8'):
    '''
        获取网页源码
    '''
    headers = {
        'User-Agent': ua.random,
    }
    r = requests.get(url, headers=headers)
    r.encoding = r.apparent_encoding
    return r.text
#     return r.text.encode(encoding).decode('unicode_escape')

In [39]:
def get_text_by_xpath(html, xpath):
    return Selector(text=html).xpath(xpath).extract()

In [10]:
def get_chinese_from_json(html):
    try:
        json_text = json.loads(html)
    except JSONDecodeError as e:
        return ''

In [11]:
def get_country_from_wiki_on_keyword(keyword):
    '''
        通过关键词，获取wiki反馈的html信息
    '''
    wiki_api = \
    '''
    https://w.bk.wjbk.site/w/api.php?
format=json&action=query&generator=search&gsrnamespace=0&gsrlimit=10&prop=pageimages|extracts&pilimit=max
&exintro&explaintext&exsentences=1&exlimit=max&origin=*&gsrsearch={}
    '''
    url = wiki_api.format(quote(keyword), 'utf-8')
    html = get_html(url)
    chinese_list = re.findall('[\u4e00-\u9fa5]+', html)
    country = get_country_from_text(chinese_list)
    if country:
        return country
    return ''
#     else:
#         return get_country_from_text(description)

In [44]:
def get_country_from_baike_on_keyword(keyword):
    '''
        通过关键词，获取wiki反馈的html信息
    '''
    baike_api = 'https://baike.baidu.com/item/{}'
    xpath = '//div[re:test(@class, "lemma-summary|lemmaWgt-lemmaTitle|basic-info|poster")]//text()'
    
    url = baike_api.format(quote(keyword), 'utf-8')
    html = get_html(url)
    text_list = get_text_by_xpath(html, xpath)
    chinese_list = re.findall('[\u4e00-\u9fa5]+', ''.join(text_list))
    country = get_country_from_text(chinese_list)
    if country:
        return country
    return ''

In [46]:
test_url = 'https://w.bk.wjbk.site/w/api.php?format=json&action=query&generator=search&gsrnamespace=0&gsrlimit=10&prop=pageimages|extracts&pilimit=max&exintro&explaintext&exsentences=1&exlimit=max&origin=*&gsrsearch=首尔'
html = get_html(test_url)
get_country_from_wiki_on_keyword('Beijing')
# '{"errno":2}' == get_country_from_baike_on_keyword('Beijing')


''